#  Predict in batch using SageMaker Batch Transform

In [ ]:
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
from sagemaker.utils import name_from_base
from sagemaker.pytorch import PyTorchModel

role = sagemaker.get_execution_role()
model_artefact = '<your-model-s3-path>'

## Create SageMaker model

In [ ]:
model = PyTorchModel(model_data=model_artefact,
                   name=name_from_base('bert-model'),
                   role=role, 
                   entry_point='predict_batch.py',
                   source_dir='source_dir',
                   framework_version='1.5.0')

## Predict

In [ ]:
model_name = model.name
test_data = 's3://sagemaker-eu-west-1-366243680492/data-processing-2020-06-28-02-22-46-897/output/preprocessed/test_batch_transform.csv'
s3_output = 's3://sagemaker-eu-west-1-366243680492/data-processing-2020-06-28-02-22-46-897/output/preprocessed'


transformer = model.transformer(
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    strategy='SingleRecord',
    assemble_with='Line',
    accept = 'text/csv',
    max_concurrent_transforms=50,
    output_path=s3_output
)

In [ ]:
transformer.transform(
    test_data,
    content_type='text/csv',
    split_type='Line',
    join_source='Input'
)
transformer.wait()